In [ ]:
#1
# Import the required libraries
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

# Load the IMDB dataset
(train_dataset, test_dataset), dataset_info = tfds.load('imdb_reviews', split=['train', 'test'], shuffle_files=True, with_info=True, as_supervised=True)

# Shuffle the data for training and create batches of text and label pairs
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Encode the text data using TextVectorization layer
encoder = TextVectorization(max_tokens=1000)
encoder.adapt(train_dataset.map(lambda text, label: text))

# Build a sequential model using tf.keras.Sequential function
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

# Train the model using train dataset
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

# Test the data using test dataset and evaluate the model by passing a sentence
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

# Predicting sentiment of a sentence
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))

if predictions[0] >= 0.0:
    print("Positive sentiment")
else:
    print("Negative sentiment")


In [ ]:
#2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt
# Download the flower dataset
tf.keras.utils.get_file('flower_photos.tgz', origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz', untar=True)

# Analyze the images in the dataset
data_dir = './flower_photos'
train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'validation')

# Get the list of all the images in the training and validation directories
train_filenames = tf.io.gfile.glob(os.path.join(train_dir, '*/*.jpg'))
validation_filenames = tf.io.gfile.glob(os.path.join(validation_dir, '*/*.jpg'))

# Print the number of images in the training and validation datasets
print('Number of training images:', len(train_filenames))
print('Number of validation images:', len(validation_filenames))
# Specify the image resolution
image_size = (224, 224)

# Split the dataset into training and validation sets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=32,
    image_size=image_size,
    shuffle=True
)

validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    validation_dir,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=32,
    image_size=image_size,
    shuffle=True
)
# Get the class names
class_names = train_ds.class_names

# Plot a sample image from each class
for i in range(len(class_names)):
    plt.figure()
    plt.imshow(tf.keras.preprocessing.image.load_img(train_filenames[i]))
    plt.title(class_names[i])
    plt.show()
# Autotune the data
train_ds = train_ds.cache().shuffle(1024).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Normalize the data
train_ds = train_ds.map(lambda x, y: (x / 255.0, y))
validation_ds = validation_ds.map(lambda x, y: (x / 255.0, y))
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')
])
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
